In [1]:
%load_ext autoreload
%autoreload 2
import datajoint as dj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pipeline.ingest import behavior as behavior_ingest
from pipeline.ingest import ephys as ephys_ingest
from pipeline import lab, get_schema_name, experiment, foraging_analysis, report, ephys, histology, psth_foraging, ccf
import pipeline.shell as shell
from pipeline import psth_foraging, foraging_model
from pipeline.plot import unit_psth

shell.logsetup('INFO')
[hh for hh in dj.list_schemas() if 'hanhou' in hh]

Connecting hanhou@datajoint.mesoscale-activity-map.org:3306


['hanhou_foraging_ccf',
 'hanhou_foraging_ephys',
 'hanhou_foraging_experiment',
 'hanhou_foraging_foraging_analysis',
 'hanhou_foraging_foraging_model',
 'hanhou_foraging_histology',
 'hanhou_foraging_ingest_behavior',
 'hanhou_foraging_ingest_ephys',
 'hanhou_foraging_ingest_histology',
 'hanhou_foraging_lab',
 'hanhou_foraging_psth',
 'hanhou_foraging_psth_foraging',
 'hanhou_foraging_publication',
 'hanhou_foraging_report',
 'hanhou_foraging_tracking',
 'hanhou_test_schema']

# Unit PSTH

## Previous pipeline

First, make sure the `psth_foraging.TrialCondition` have the correct conditions. <br> Note that I separate foraging ephys analysis to another new schema `psth_foraging`.

In [ ]:
from pipeline import psth_foraging

In [ ]:
psth_foraging.TrialCondition()

In [ ]:
psth_foraging.TrialCondition.get_trials('foraging_L_hit_noearlylick').fetch(format='frame', limit=10)

Here I need to run `foraging_populate.py` to do parallel population of `psth_foraging.UnitPsth`

Check progress of populating PSTH

In [ ]:
schema = dj.schema(get_schema_name('psth_foraging'))
schema.jobs

In [ ]:
schema.jobs.fetch('key')

In [ ]:
length = len(psth_foraging.UnitPsth())
ratio = length / len(psth_foraging.UnitPsth.key_source)
print(f'ephys.Unit:{length}, {ratio:.2%}')

In [ ]:
schema.jobs.delete()

In [ ]:
dj.kill()

After populating PSTH (with some bug fixes), single unit PSTH can be plotted

In [ ]:
experiment.Session & 'session_date = "2021-04-29"'

In [ ]:
from pipeline.plot import unit_psth

In [ ]:
unit = (ephys.Unit() & (experiment.Session & 'session_date = "2021-04-29"') & 'insertion_number=1' & 'unit_uid=299').fetch1("KEY")
_ = unit_psth.plot_unit_psth_foraging(unit)

## New flexible PSTHs

In [ ]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() & (experiment.Session & 'session_date = "2021-04-18"' & 'subject_id = 473361') & {'insertion_number': imec + 1, 'unit_uid': unit}).fetch1("KEY")

### PSTH grouped by choice and outcome

In [ ]:
align_types=['trial_start', 'go_cue', 'first_lick_after_go_cue', 'iti_start', 'next_trial_start']

In [ ]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

In [ ]:
_ = unit_psth.plot_unit_psth_choice_outcome(unit_key, align_types)

In [ ]:
_ = unit_psth.plot_unit_psth_choice_outcome(unit_key, align_types=['iti_start', 'next_trial_start'], if_raster=False)

### PSTH grouped by value quantiles

In [ ]:
best_model = (foraging_model.FittedSessionModelComparison.BestModel & unit_key & 'model_comparison_idx = 1').fetch1('best_aic')
print(best_model)

In [ ]:
align_types=['trial_start', 'go_cue', 'first_lick_after_go_cue', 'iti_start', 'next_trial_start']
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=best_model, align_types=align_types, latent_variable='contra_action_value')

In [ ]:
align_types=['trial_start', 'first_lick_after_go_cue', 'iti_start', 'next_trial_start']
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=best_model, align_types=align_types, latent_variable='ipsi_action_value')

In [ ]:
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=best_model, align_types=align_types, latent_variable='relative_action_value_ic')

In [ ]:
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=best_model, align_types=align_types, latent_variable='total_action_value')

### More example neurons

####  PL

In [ ]:
date, imec, unit = '2021-04-18', 0, 541
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=5)

In [ ]:
date, imec, unit = '2021-04-18', 0, 484
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

#### ALM

In [ ]:
date, imec, unit = '2021-04-29', 0, 501
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

In [ ]:
date, imec, unit = '2021-04-16', 0, 978
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

In [ ]:
date, imec, unit = '2021-04-29', 1, 362
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10, side='ipsi')

#### Thalamus

In [ ]:
date, imec, unit = '2021-06-11', 1, 75
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

In [ ]:
date, imec, unit = '2021-06-11', 1, 310
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

#### Striatum

In [ ]:
date, imec, unit = '2021-04-29', 0, 113
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10, side='ipsi')

####  Midbrain

These two should be in VTA

In [ ]:
date, imec, unit = '2021-06-14', 0, 22
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

In [ ]:
date, imec, unit = '2021-06-14', 0, 32
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

#### Ventral Pallidum

In [ ]:
date, imec, unit = '2021-06-09', 1, 175
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

In [ ]:
date, imec, unit = '2021-06-09', 1, 125
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

#### RSC

In [ ]:
date, imec, unit = '2021-06-12', 0, 168
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

#### OFC

In [ ]:
date, imec, unit = '2021-04-29', 0, 299
unit_key = (ephys.Unit() * experiment.Session & {'session_date': date, 'subject_id': 473361, 'insertion_number':imec + 1, 'unit_uid': unit}).fetch1("KEY")

_ = unit_psth.plot_unit_psth_choice_outcome(unit_key)
_ = unit_psth.plot_unit_psth_latent_variable_quantile(unit_key, model_id=10)

# Unit selectivity 

## Preparation

In [2]:
psth_foraging.UnitTrialPeriodFiring.key_source

subject_id institution 6 digit animal ID,session session number,insertion_number,clustering_method,unit,trial trial number (1-based indexing),unit_uid unique across sessions/animals,unit_quality,probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name,electrode_group electrode group,"electrode electrode index, starts at 1","unit_posx (um) estimated x position of the unit relative to probe's tip (0,0)","unit_posy (um) estimated y position of the unit relative to probe's tip (0,0)",spike_times (s) from the start of the first data point used in clustering,spike_sites array of electrode associated with each spike,spike_depths (um) array of depths associated with each spike,unit_amp,unit_snr,waveform average spike waveform,trial_uid unique across sessions/animals,start_time (s) relative to session beginning,stop_time (s) relative to session beginning
473361,42,1,kilosort2,0,1,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,-3.9540,9.7360
473361,42,1,kilosort2,0,2,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,9.7360,15.6280
473361,42,1,kilosort2,0,3,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,15.6280,29.4260
473361,42,1,kilosort2,0,4,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,29.4260,43.7260
473361,42,1,kilosort2,0,5,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,43.7260,58.3080
473361,42,1,kilosort2,0,6,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,58.3080,75.2360
473361,42,1,kilosort2,0,7,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,75.2360,87.0090
473361,42,1,kilosort2,0,8,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,87.0090,98.2550
473361,42,1,kilosort2,0,9,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,98.2550,108.1200
473361,42,1,kilosort2,0,10,0,multi,neuropixels 1.0 - 3B,1-384,0,137,27.0,1360.0,=BLOB=,=BLOB=,=BLOB=,16.685151356993735,0.9454203275476792,=BLOB=,30444,108.1200,117.0940


subject_id           : int                          # institution 6 digit animal ID
session              : smallint                     # session number
trial                : smallint                     # trial number (1-based indexing)
---
trial_uid            : int                          # unique across sessions/animals
start_time           : decimal(9,4)                 # (s) relative to session beginning
stop_time            : decimal(9,4)                 # (s) relative to session beginning